# Amber Tutorial: Local Execution

This Notebook demonstrates how to set up and run a basic Amber simulations.
The studied system is the AMDase enzyme, a decerboxylate, with a bound simple substrate. The substrate have four different possible binding modes, which differ in whether the pro-S or pro-R prochiral carboxylate group is cleaved, and also, whether the reaction progresses with an inversion or retention mechanism. In this tutorial, we are going to run classical Molecular Dynamics simulations to compare the feasibilty of the four binding modes. The required topology and coordinate files are made available.

In [1]:
%pip install -qU "airavata-python-sdk[notebook]"
import airavata_jupyter_magic

%authenticate
%request_runtime hpc_gpu --file=cybershuttle.yml --walltime=60 --use=expanse:gpu-shared
%wait_for_runtime hpc_gpu --live
%switch_runtime hpc_gpu

Note: you may need to restart the kernel to use updated packages.

Loaded airavata_jupyter_magic (2.1.4.post1) 
(current runtime = local)

  %authenticate                              -- Authenticate to access high-performance runtimes.
  %request_runtime <rt> [args]               -- Request a runtime named <rt> with configuration <args>.
                                                Call multiple times to request multiple runtimes.
  %restart_runtime <rt>                      -- Restart runtime <rt> if it hangs. This will clear all variables.
  %stop_runtime <rt>                         -- Stop runtime <rt> when no longer needed.
  %wait_for_runtime <rt>                     -- Wait for runtime <rt> to be ready.
  %switch_runtime <rt>                       -- Switch the active runtime to <rt>. All subsequent cells will run here.
  %%run_on <rt>                              -- Force a cell to always execute on <rt>, regardless of the active runtime.
  %stat_runtime <rt>               

Output()

Authenticated.

Requesting runtime=hpc_gpu...
[expanse:gpu-shared, 60 Minutes, 1 Node(s), 4 CPU(s), 0 GPU(s), 4096 MB RAM, 1024 MB VRAM]
* modules=[]
* libraries=['python=3.10', 'pip', 'numpy', 'tqdm', 'rich']
* pip=[]
* mounts=[]
Requested runtime=hpc_gpu
Request successful: runtime=hpc_gpu


Output()

local:/tmp/connection_i2h16xbt.json --> hpc_gpu:connection_i2h16xbt.json... [200]
started proc_name=hpc_gpu_kernel on rt=hpc_gpu. pid=3739196
forwarding ports=[29410, 29411, 29412, 29413, 29414]
hpc_gpu:29410 -> access via 18.118.140.230:10010
hpc_gpu:29411 -> access via 18.118.140.230:10011
hpc_gpu:29412 -> access via 18.118.140.230:10012
hpc_gpu:29413 -> access via 18.118.140.230:10013
hpc_gpu:29414 -> access via 18.118.140.230:10014
18.118.140.230
started ipykernel client for hpc_gpu
Remote Jupyter kernel launched and connected for runtime=hpc_gpu.
Switched to runtime=hpc_gpu.


# 1.1 Binding mode A: minimization

The first binding mode, A (pro-S cleavage, inversion mechanism) will be simulated in separate steps, starting with the minimization of the structure.

In [16]:
%copy_data source=local:proS_inv/1_md target=hpc_gpu:.

copying local:proS_inv/1_md to hpc_gpu:.
local:proS_inv/1_md/equil11npt.in --> hpc_gpu:equil11npt.in... [200]
local:proS_inv/1_md/equil7npt.in --> hpc_gpu:equil7npt.in... [200]
local:proS_inv/1_md/heat2nvt.in --> hpc_gpu:heat2nvt.in... [200]
local:proS_inv/1_md/distance_cpptraj.in --> hpc_gpu:distance_cpptraj.in... [200]
local:proS_inv/1_md/heat5nvt.in --> hpc_gpu:heat5nvt.in... [200]
local:proS_inv/1_md/dist_cpptraj_proR_ret_3.in --> hpc_gpu:dist_cpptraj_proR_ret_3.in... [200]
local:proS_inv/1_md/heat3npt.in --> hpc_gpu:heat3npt.in... [200]
local:proS_inv/1_md/dist_cpptraj_proS_inv_1.in --> hpc_gpu:dist_cpptraj_proS_inv_1.in... [200]
local:proS_inv/1_md/3ixl_CLGIPL_methyl_proR_ret_complex.top --> hpc_gpu:3ixl_CLGIPL_methyl_proR_ret_complex.top... [200]
local:proS_inv/1_md/3ixl_CLGIPL_methyl_proS_inv_complex.top --> hpc_gpu:3ixl_CLGIPL_methyl_proS_inv_complex.top... [200]
local:proS_inv/1_md/production1.in --> hpc_gpu:production1.in... [200]
local:proS_inv/1_md/equil8npt.in --> hpc_gpu

In [17]:
%%bash

module load gpu/0.17.3b  gcc/8.4.0/xiuwkua  openmpi/4.1.3/v2ei3ge amber/22/ulauqq7-omp

pmemd.cuda_SPFP \
                       -O -i min1.in \
                       -o  3ixl_CLGIPL_methyl_proS_inv_complex_1opt.log \
                       -c  3ixl_CLGIPL_methyl_proS_inv_complex.crd \
                       -p  3ixl_CLGIPL_methyl_proS_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_inv_complex_1opt.rst \
                       -x  3ixl_CLGIPL_methyl_proS_inv_complex_1opt.crd \
                       -ref 3ixl_CLGIPL_methyl_proS_inv_complex.crd

executing cell on hpc_gpu...
waiting for cell to finish on hpc_gpu...



Lmod is automatically replacing "gcc/10.2.0/npcyll4" with "gcc/8.4.0/xiuwkua".



cell finished on hpc_gpu.


Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_OVERFLOW_FLAG IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


In [18]:
! ls -lrt | grep 3ixl

executing cell on hpc_gpu...
waiting for cell to finish on hpc_gpu...
-rw-r--r-- 1 gridchem uic151  5782173 May  7 12:20 3ixl_CLGIPL_methyl_proR_ret_complex.top
-rw-r--r-- 1 gridchem uic151  5782650 May  7 12:20 3ixl_CLGIPL_methyl_proS_inv_complex.top
-rw-r--r-- 1 gridchem uic151  1103014 May  7 12:20 3ixl_CLGIPL_methyl_proS_inv_complex.crd
-rw-r--r-- 1 gridchem uic151   726052 May  7 12:21 3ixl_CLGIPL_methyl_proS_inv_complex_1opt.rst
-rw-r--r-- 1 gridchem uic151    15723 May  7 12:21 3ixl_CLGIPL_methyl_proS_inv_complex_1opt.log
cell finished on hpc_gpu.


# 1.2 Binding mode A: heating

In the next two steps, we heat our system to 300 K, with restraints on the protein heavy atoms.

In [ ]:
%%bash

module load gpu/0.17.3b  gcc/8.4.0/xiuwkua  openmpi/4.1.3/v2ei3ge amber/22/ulauqq7-omp

pmemd.cuda_SPFP \
                       -O -i heat2nvt.in \
                       -o  3ixl_CLGIPL_methyl_proS_inv_complex_2heat.log \
                       -c  3ixl_CLGIPL_methyl_proS_inv_complex_1opt.rst \
                       -p  3ixl_CLGIPL_methyl_proS_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_inv_complex_2heat.rst \
                       -x  3ixl_CLGIPL_methyl_proS_inv_complex_2heat.crd \
                       -ref 3ixl_CLGIPL_methyl_proS_inv_complex_1opt.rst

pmemd.cuda_SPFP \
                       -O -i  heat3npt.in \
                       -o  3ixl_CLGIPL_methyl_proS_inv_complex_3heat.log \
                       -c  3ixl_CLGIPL_methyl_proS_inv_complex_2heat.rst \
                       -p  3ixl_CLGIPL_methyl_proS_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_inv_complex_3heat.rst \
                       -x  3ixl_CLGIPL_methyl_proS_inv_complex_3heat.crd \
                       -ref  3ixl_CLGIPL_methyl_proS_inv_complex_2heat.rst


executing cell on hpc_gpu...
waiting for cell to finish on hpc_gpu...



Lmod is automatically replacing "gcc/10.2.0/npcyll4" with "gcc/8.4.0/xiuwkua".

Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_OVERFLOW_FLAG IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


# 1.3 Binding mode A: minimization 2.

In the next two steps, we minimize our system for a second time.

In [ ]:
%%bash

module load gpu/0.17.3b  gcc/8.4.0/xiuwkua  openmpi/4.1.3/v2ei3ge amber/22/ulauqq7-omp

pmemd.cuda_SPFP \
                       -O -i min4.in \
                       -o  3ixl_CLGIPL_methyl_proS_inv_complex_4opt.log \
                       -c  3ixl_CLGIPL_methyl_proS_inv_complex_3heat.rst \
                       -p  3ixl_CLGIPL_methyl_proS_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_inv_complex_4opt.rst \
                       -x  3ixl_CLGIPL_methyl_proS_inv_complex_4opt.crd \
                       -ref 3ixl_CLGIPL_methyl_proS_inv_complex_3heat.rst


# 1.4 Binding mode A: heating

In the next two steps, we heat our system to 300 K, with weak restraints on the C-alpha atoms.

In [ ]:
%%bash

module load gpu/0.17.3b  gcc/8.4.0/xiuwkua  openmpi/4.1.3/v2ei3ge amber/22/ulauqq7-omp

pmemd.cuda_SPFP \
                       -O -i heat5nvt.in \
                       -o  3ixl_CLGIPL_methyl_proS_inv_complex_5heat.log \
                       -c  3ixl_CLGIPL_methyl_proS_inv_complex_4opt.rst \
                       -p  3ixl_CLGIPL_methyl_proS_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_inv_complex_5heat.rst \
                       -x  3ixl_CLGIPL_methyl_proS_inv_complex_5heat.crd \
                       -ref 3ixl_CLGIPL_methyl_proS_inv_complex_4opt.rst

pmemd.cuda_SPFP \
                       -O -i  heat6npt.in \
                       -o  3ixl_CLGIPL_methyl_proS_inv_complex_6heat.log \
                       -c  3ixl_CLGIPL_methyl_proS_inv_complex_5heat.rst \
                       -p  3ixl_CLGIPL_methyl_proS_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_inv_complex_6heat.rst \
                       -x  3ixl_CLGIPL_methyl_proS_inv_complex_6heat.crd \
                       -ref  3ixl_CLGIPL_methyl_proS_inv_complex_5heat.rst


# 1.5 Binding mode A: equilibration

In the following steps, we run equilibration steps, with increasingly smaller restraints of the protein C-alpha atoms, with the final equilibration run without restraints.

In [ ]:
%%bash

module load gpu/0.17.3b  gcc/8.4.0/xiuwkua  openmpi/4.1.3/v2ei3ge amber/22/ulauqq7-omp

pmemd.cuda_SPFP \
                       -O -i  equil7npt.in \
                       -o  3ixl_CLGIPL_methyl_proS_inv_complex_7equil.log \
                       -c  3ixl_CLGIPL_methyl_proS_inv_complex_6heat.rst \
                       -p  3ixl_CLGIPL_methyl_proS_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_inv_complex_7equil.rst \
                       -x  3ixl_CLGIPL_methyl_proS_inv_complex_7equil.crd \
                       -ref 3ixl_CLGIPL_methyl_proS_inv_complex_6heat.rst
step=7

while [ $step -le 11 ]
do
step_pos=$(($step+1))

pmemd.cuda_SPFP \
                       -O -i  equil${step_pos}npt.in \
                       -o  3ixl_CLGIPL_methyl_proS_inv_complex_${step_pos}equil.log \
                       -c  3ixl_CLGIPL_methyl_proS_inv_complex_${step}equil.rst \
                       -p  3ixl_CLGIPL_methyl_proS_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_inv_complex_${step_pos}equil.rst \
                       -x  3ixl_CLGIPL_methyl_proS_inv_complex_${step_pos}equil.crd \
                       -ref 3ixl_CLGIPL_methyl_proS_inv_complex_${step}equil.rst
step=$((step+1))
done


# 1.6 Binding mode A: production run

In the final molecular dynamics step, we run a longer production run, with frequent sampling, for analysis.

In [ ]:
%%bash

module load gpu/0.17.3b  gcc/8.4.0/xiuwkua  openmpi/4.1.3/v2ei3ge amber/22/ulauqq7-omp

pmemd.cuda_SPFP \
                       -O -i  production1.in \
                       -o  3ixl_CLGIPL_methyl_proS_inv_complex_PRODUCTION_1.log \
                       -c  3ixl_CLGIPL_methyl_proS_inv_complex_12equil.rst \
                       -p  3ixl_CLGIPL_methyl_proS_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_inv_complex_PRODUCTION_1.rst \
                       -x  3ixl_CLGIPL_methyl_proS_inv_complex_PRODUCTION_1.crd \
                       -ref 3ixl_CLGIPL_methyl_proS_inv_complex_12equil.rst


# 2 Binding mode B: Complete simulation protocol

After completing the individual simulation steps for Binding mode A, we continue with Binding mode B (pro-S cleavage, retention mechanism). For this, the complete protocol is run in one go.

In [ ]:
%%bash

module load gpu/0.17.3b  gcc/8.4.0/xiuwkua  openmpi/4.1.3/v2ei3ge amber/22/ulauqq7-omp

pmemd.cuda_SPFP \
                       -O -i min1.in \
                       -o  3ixl_CLGIPL_methyl_proS_ret_complex_1opt.log \
                       -c  3ixl_CLGIPL_methyl_proS_ret_complex.crd \
                       -p  3ixl_CLGIPL_methyl_proS_ret_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_ret_complex_1opt.rst \
                       -x  3ixl_CLGIPL_methyl_proS_ret_complex_1opt.crd \
                       -ref 3ixl_CLGIPL_methyl_proS_ret_complex.crd

pmemd.cuda_SPFP \
                       -O -i heat2nvt.in \
                       -o  3ixl_CLGIPL_methyl_proS_ret_complex_2heat.log \
                       -c  3ixl_CLGIPL_methyl_proS_ret_complex_1opt.rst \
                       -p  3ixl_CLGIPL_methyl_proS_ret_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_ret_complex_2heat.rst \
                       -x  3ixl_CLGIPL_methyl_proS_ret_complex_2heat.crd \
                       -ref 3ixl_CLGIPL_methyl_proS_ret_complex_1opt.rst

pmemd.cuda_SPFP \
                       -O -i  heat3npt.in \
                       -o  3ixl_CLGIPL_methyl_proS_ret_complex_3heat.log \
                       -c  3ixl_CLGIPL_methyl_proS_ret_complex_2heat.rst \
                       -p  3ixl_CLGIPL_methyl_proS_ret_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_ret_complex_3heat.rst \
                       -x  3ixl_CLGIPL_methyl_proS_ret_complex_3heat.crd \
                       -ref  3ixl_CLGIPL_methyl_proS_ret_complex_2heat.rst

pmemd.cuda_SPFP \
                       -O -i min4.in \
                       -o  3ixl_CLGIPL_methyl_proS_ret_complex_4opt.log \
                       -c  3ixl_CLGIPL_methyl_proS_ret_complex_3heat.rst \
                       -p  3ixl_CLGIPL_methyl_proS_ret_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_ret_complex_4opt.rst \
                       -x  3ixl_CLGIPL_methyl_proS_ret_complex_4opt.crd \
                       -ref 3ixl_CLGIPL_methyl_proS_ret_complex_3heat.rst

pmemd.cuda_SPFP \
                       -O -i heat5nvt.in \
                       -o  3ixl_CLGIPL_methyl_proS_ret_complex_5heat.log \
                       -c  3ixl_CLGIPL_methyl_proS_ret_complex_4opt.rst \
                       -p  3ixl_CLGIPL_methyl_proS_ret_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_ret_complex_5heat.rst \
                       -x  3ixl_CLGIPL_methyl_proS_ret_complex_5heat.crd \
                       -ref 3ixl_CLGIPL_methyl_proS_ret_complex_4opt.rst

pmemd.cuda_SPFP \
                       -O -i  heat6npt.in \
                       -o  3ixl_CLGIPL_methyl_proS_ret_complex_6heat.log \
                       -c  3ixl_CLGIPL_methyl_proS_ret_complex_5heat.rst \
                       -p  3ixl_CLGIPL_methyl_proS_ret_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_ret_complex_6heat.rst \
                       -x  3ixl_CLGIPL_methyl_proS_ret_complex_6heat.crd \
                       -ref  3ixl_CLGIPL_methyl_proS_ret_complex_5heat.rst

pmemd.cuda_SPFP \
                       -O -i  equil7npt.in \
                       -o  3ixl_CLGIPL_methyl_proS_ret_complex_7equil.log \
                       -c  3ixl_CLGIPL_methyl_proS_ret_complex_6heat.rst \
                       -p  3ixl_CLGIPL_methyl_proS_ret_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_ret_complex_7equil.rst \
                       -x  3ixl_CLGIPL_methyl_proS_ret_complex_7equil.crd \
                       -ref 3ixl_CLGIPL_methyl_proS_ret_complex_6heat.rst
step=7

while [ $step -le 11 ]
do
step_pos=$(($step+1))

pmemd.cuda_SPFP \
                       -O -i  equil${step_pos}npt.in \
                       -o  3ixl_CLGIPL_methyl_proS_ret_complex_${step_pos}equil.log \
                       -c  3ixl_CLGIPL_methyl_proS_ret_complex_${step}equil.rst \
                       -p  3ixl_CLGIPL_methyl_proS_ret_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_ret_complex_${step_pos}equil.rst \
                       -x  3ixl_CLGIPL_methyl_proS_ret_complex_${step_pos}equil.crd \
                       -ref 3ixl_CLGIPL_methyl_proS_ret_complex_${step}equil.rst
step=$((step+1))
done

pmemd.cuda_SPFP \
                       -O -i  production1.in \
                       -o  3ixl_CLGIPL_methyl_proS_ret_complex_PRODUCTION_1.log \
                       -c  3ixl_CLGIPL_methyl_proS_ret_complex_12equil.rst \
                       -p  3ixl_CLGIPL_methyl_proS_ret_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_ret_complex_PRODUCTION_1.rst \
                       -x  3ixl_CLGIPL_methyl_proS_ret_complex_PRODUCTION_1.crd \
                       -ref 3ixl_CLGIPL_methyl_proS_ret_complex_12equil.rst


# 3 Binding mode C: Complete simulation protocol

We continue the simulations with Binding mode C (pro-R cleavage, inversion mechanism).

In [ ]:
%%bash

module load gpu/0.17.3b  gcc/8.4.0/xiuwkua  openmpi/4.1.3/v2ei3ge amber/22/ulauqq7-omp

pmemd.cuda_SPFP \
                       -O -i min1.in \
                       -o  3ixl_CLGIPL_methyl_proR_inv_complex_1opt.log \
                       -c  3ixl_CLGIPL_methyl_proR_inv_complex.crd \
                       -p  3ixl_CLGIPL_methyl_proR_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proR_inv_complex_1opt.rst \
                       -x  3ixl_CLGIPL_methyl_proR_inv_complex_1opt.crd \
                       -ref 3ixl_CLGIPL_methyl_proR_inv_complex.crd

pmemd.cuda_SPFP \
                       -O -i heat2nvt.in \
                       -o  3ixl_CLGIPL_methyl_proR_inv_complex_2heat.log \
                       -c  3ixl_CLGIPL_methyl_proR_inv_complex_1opt.rst \
                       -p  3ixl_CLGIPL_methyl_proR_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proR_inv_complex_2heat.rst \
                       -x  3ixl_CLGIPL_methyl_proR_inv_complex_2heat.crd \
                       -ref 3ixl_CLGIPL_methyl_proR_inv_complex_1opt.rst

pmemd.cuda_SPFP \
                       -O -i  heat3npt.in \
                       -o  3ixl_CLGIPL_methyl_proR_inv_complex_3heat.log \
                       -c  3ixl_CLGIPL_methyl_proR_inv_complex_2heat.rst \
                       -p  3ixl_CLGIPL_methyl_proR_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proR_inv_complex_3heat.rst \
                       -x  3ixl_CLGIPL_methyl_proR_inv_complex_3heat.crd \
                       -ref  3ixl_CLGIPL_methyl_proR_inv_complex_2heat.rst

pmemd.cuda_SPFP \
                       -O -i min4.in \
                       -o  3ixl_CLGIPL_methyl_proR_inv_complex_4opt.log \
                       -c  3ixl_CLGIPL_methyl_proR_inv_complex_3heat.rst \
                       -p  3ixl_CLGIPL_methyl_proR_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proR_inv_complex_4opt.rst \
                       -x  3ixl_CLGIPL_methyl_proR_inv_complex_4opt.crd \
                       -ref 3ixl_CLGIPL_methyl_proR_inv_complex_3heat.rst

pmemd.cuda_SPFP \
                       -O -i heat5nvt.in \
                       -o  3ixl_CLGIPL_methyl_proR_inv_complex_5heat.log \
                       -c  3ixl_CLGIPL_methyl_proR_inv_complex_4opt.rst \
                       -p  3ixl_CLGIPL_methyl_proR_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proR_inv_complex_5heat.rst \
                       -x  3ixl_CLGIPL_methyl_proR_inv_complex_5heat.crd \
                       -ref 3ixl_CLGIPL_methyl_proR_inv_complex_4opt.rst

pmemd.cuda_SPFP \
                       -O -i  heat6npt.in \
                       -o  3ixl_CLGIPL_methyl_proR_inv_complex_6heat.log \
                       -c  3ixl_CLGIPL_methyl_proR_inv_complex_5heat.rst \
                       -p  3ixl_CLGIPL_methyl_proR_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proR_inv_complex_6heat.rst \
                       -x  3ixl_CLGIPL_methyl_proR_inv_complex_6heat.crd \
                       -ref  3ixl_CLGIPL_methyl_proR_inv_complex_5heat.rst

pmemd.cuda_SPFP \
                       -O -i  equil7npt.in \
                       -o  3ixl_CLGIPL_methyl_proR_inv_complex_7equil.log \
                       -c  3ixl_CLGIPL_methyl_proR_inv_complex_6heat.rst \
                       -p  3ixl_CLGIPL_methyl_proR_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proR_inv_complex_7equil.rst \
                       -x  3ixl_CLGIPL_methyl_proR_inv_complex_7equil.crd \
                       -ref 3ixl_CLGIPL_methyl_proR_inv_complex_6heat.rst
step=7

while [ $step -le 11 ]
do
step_pos=$(($step+1))

pmemd.cuda_SPFP \
                       -O -i  equil${step_pos}npt.in \
                       -o  3ixl_CLGIPL_methyl_proR_inv_complex_${step_pos}equil.log \
                       -c  3ixl_CLGIPL_methyl_proR_inv_complex_${step}equil.rst \
                       -p  3ixl_CLGIPL_methyl_proR_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proR_inv_complex_${step_pos}equil.rst \
                       -x  3ixl_CLGIPL_methyl_proR_inv_complex_${step_pos}equil.crd \
                       -ref 3ixl_CLGIPL_methyl_proR_inv_complex_${step}equil.rst
step=$((step+1))
done

pmemd.cuda_SPFP \
                       -O -i  production1.in \
                       -o  3ixl_CLGIPL_methyl_proR_inv_complex_PRODUCTION_1.log \
                       -c  3ixl_CLGIPL_methyl_proR_inv_complex_12equil.rst \
                       -p  3ixl_CLGIPL_methyl_proR_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proR_inv_complex_PRODUCTION_1.rst \
                       -x  3ixl_CLGIPL_methyl_proR_inv_complex_PRODUCTION_1.crd \
                       -ref 3ixl_CLGIPL_methyl_proR_inv_complex_12equil.rst


# 4 Binding mode D: Complete simulation protocol

We continue the simulations with Binding mode D (pro-R cleavage, retention mechanism).

In [ ]:
%%bash

module load gpu/0.17.3b  gcc/8.4.0/xiuwkua  openmpi/4.1.3/v2ei3ge amber/22/ulauqq7-omp

pmemd.cuda_SPFP \
                       -O -i min1.in \
                       -o  3ixl_CLGIPL_methyl_proR_ret_complex_1opt.log \
                       -c  3ixl_CLGIPL_methyl_proR_ret_complex.crd \
                       -p  3ixl_CLGIPL_methyl_proR_ret_complex.top \
                       -r  3ixl_CLGIPL_methyl_proR_ret_complex_1opt.rst \
                       -x  3ixl_CLGIPL_methyl_proR_ret_complex_1opt.crd \
                       -ref 3ixl_CLGIPL_methyl_proR_ret_complex.crd

pmemd.cuda_SPFP \
                       -O -i heat2nvt.in \
                       -o  3ixl_CLGIPL_methyl_proR_ret_complex_2heat.log \
                       -c  3ixl_CLGIPL_methyl_proR_ret_complex_1opt.rst \
                       -p  3ixl_CLGIPL_methyl_proR_ret_complex.top \
                       -r  3ixl_CLGIPL_methyl_proR_ret_complex_2heat.rst \
                       -x  3ixl_CLGIPL_methyl_proR_ret_complex_2heat.crd \
                       -ref 3ixl_CLGIPL_methyl_proR_ret_complex_1opt.rst

pmemd.cuda_SPFP \
                       -O -i  heat3npt.in \
                       -o  3ixl_CLGIPL_methyl_proR_ret_complex_3heat.log \
                       -c  3ixl_CLGIPL_methyl_proR_ret_complex_2heat.rst \
                       -p  3ixl_CLGIPL_methyl_proR_ret_complex.top \
                       -r  3ixl_CLGIPL_methyl_proR_ret_complex_3heat.rst \
                       -x  3ixl_CLGIPL_methyl_proR_ret_complex_3heat.crd \
                       -ref  3ixl_CLGIPL_methyl_proR_ret_complex_2heat.rst

pmemd.cuda_SPFP \
                       -O -i min4.in \
                       -o  3ixl_CLGIPL_methyl_proR_ret_complex_4opt.log \
                       -c  3ixl_CLGIPL_methyl_proR_ret_complex_3heat.rst \
                       -p  3ixl_CLGIPL_methyl_proR_ret_complex.top \
                       -r  3ixl_CLGIPL_methyl_proR_ret_complex_4opt.rst \
                       -x  3ixl_CLGIPL_methyl_proR_ret_complex_4opt.crd \
                       -ref 3ixl_CLGIPL_methyl_proR_ret_complex_3heat.rst

pmemd.cuda_SPFP \
                       -O -i heat5nvt.in \
                       -o  3ixl_CLGIPL_methyl_proR_ret_complex_5heat.log \
                       -c  3ixl_CLGIPL_methyl_proR_ret_complex_4opt.rst \
                       -p  3ixl_CLGIPL_methyl_proR_ret_complex.top \
                       -r  3ixl_CLGIPL_methyl_proR_ret_complex_5heat.rst \
                       -x  3ixl_CLGIPL_methyl_proR_ret_complex_5heat.crd \
                       -ref 3ixl_CLGIPL_methyl_proR_ret_complex_4opt.rst

pmemd.cuda_SPFP \
                       -O -i  heat6npt.in \
                       -o  3ixl_CLGIPL_methyl_proR_ret_complex_6heat.log \
                       -c  3ixl_CLGIPL_methyl_proR_ret_complex_5heat.rst \
                       -p  3ixl_CLGIPL_methyl_proR_ret_complex.top \
                       -r  3ixl_CLGIPL_methyl_proR_ret_complex_6heat.rst \
                       -x  3ixl_CLGIPL_methyl_proR_ret_complex_6heat.crd \
                       -ref  3ixl_CLGIPL_methyl_proR_ret_complex_5heat.rst
pmemd.cuda_SPFP \
                       -O -i  equil7npt.in \
                       -o  3ixl_CLGIPL_methyl_proS_inv_complex_7equil.log \
                       -c  3ixl_CLGIPL_methyl_proS_inv_complex_6heat.rst \
                       -p  3ixl_CLGIPL_methyl_proS_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_inv_complex_7equil.rst \
                       -x  3ixl_CLGIPL_methyl_proS_inv_complex_7equil.crd \
                       -ref 3ixl_CLGIPL_methyl_proS_inv_complex_6heat.rst
step=7

while [ $step -le 11 ]
do
step_pos=$(($step+1))

pmemd.cuda_SPFP \
                       -O -i  equil${step_pos}npt.in \
                       -o  3ixl_CLGIPL_methyl_proS_inv_complex_${step_pos}equil.log \
                       -c  3ixl_CLGIPL_methyl_proS_inv_complex_${step}equil.rst \
                       -p  3ixl_CLGIPL_methyl_proS_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_inv_complex_${step_pos}equil.rst \
                       -x  3ixl_CLGIPL_methyl_proS_inv_complex_${step_pos}equil.crd \
                       -ref 3ixl_CLGIPL_methyl_proS_inv_complex_${step}equil.rst
step=$((step+1))
done

pmemd.cuda_SPFP \
                       -O -i  production1.in \
                       -o  3ixl_CLGIPL_methyl_proS_inv_complex_PRODUCTION_1.log \
                       -c  3ixl_CLGIPL_methyl_proS_inv_complex_12equil.rst \
                       -p  3ixl_CLGIPL_methyl_proS_inv_complex.top \
                       -r  3ixl_CLGIPL_methyl_proS_inv_complex_PRODUCTION_1.rst \
                       -x  3ixl_CLGIPL_methyl_proS_inv_complex_PRODUCTION_1.crd \
                       -ref 3ixl_CLGIPL_methyl_proS_inv_complex_12equil.rst


# 5 Analysis of the simulations

After the production runs are completed, we continue with the analysis of our simulations, with particular focus on the distance of the substrate central atom from the catalytic site. To carry out the analysis, we apply the cpptraj script of AmberTools, on some of our available trajectories. After completion, we calculate the average and SD of individual simulations.

In [ ]:
%copy_data source=local:proS_inv/1_md/3ixl_CLGIPL_methyl_proR_ret_complex.top target=hpc_gpu:.
%copy_data source=local:proS_inv/1_md/dist_cpptraj_proR_ret_1.in target=hpc_gpu:.
%copy_data source=local:cybershuttle_data/amber-trajectory-analysis/3ixl_CLGIPL_methyl_proR_ret_complex_PRODUCTION_1_first_100.crd target=hpc_gpu:.
%copy_data source=local:proS_inv/1_md/dist_cpptraj_proS_inv_1.in target=hpc_gpu:.
%copy_data source=local:cybershuttle_data/amber-trajectory-analysis/3ixl_CLGIPL_methyl_proS_inv_complex_PRODUCTION_1_first_100.crd target=hpc_gpu:.

In [ ]:
%%bash

module load gpu/0.17.3b  gcc/8.4.0/xiuwkua  openmpi/4.1.3/v2ei3ge amber/22/ulauqq7-omp

cpptraj -i dist_cpptraj_proR_ret_1.in
tail -n 100 proR_ret_dist_1.dat | awk 'FNR > 1 {sum += $2; sumsq += $2^2; count++} END {mean = sum/count; sd = sqrt(sumsq/count - mean^2); print "Mean:", mean, "SD:", sd}'

In [ ]:
%%bash

module load gpu/0.17.3b  gcc/8.4.0/xiuwkua  openmpi/4.1.3/v2ei3ge amber/22/ulauqq7-omp

cpptraj -i dist_cpptraj_proS_inv_1.in
tail -n 100 proS_inv_dist_1.dat | awk 'FNR > 1 {sum += $2; sumsq += $2^2; count++} END {mean = sum/count; sd = sqrt(sumsq/count - mean^2); print "Mean:", mean, "SD:", sd}'

In [ ]:
%stop_runtime hpc_gpu